In [2]:
from PIL import Image, ImageDraw, ImageFilter
import random
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Convolution2D,MaxPooling2D,AveragePooling2D,Flatten,Dropout
from tensorflow.keras import optimizers
from tqdm import tqdm
import matplotlib.patches as patches
plt.rcParams["figure.figsize"] = (8,6)


In [3]:
canvas_size=(300,300)
sign_size_min=100
sign_size_max=150
samples=3000
test_samples=100

In [4]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]

In [5]:
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

## Preprocessing

#### Histograms

In [6]:
row_train=np.zeros((samples-test_samples, canvas_size[1], 3, 1))
for i in range(samples-test_samples):
    for j in range(canvas_size[1]):
        for c in range(3):
            row_train[i,j,c]=np.sum(x_train[i,j,:,c])
            
row_test=np.zeros((test_samples, canvas_size[1], 3, 1))
for i in range(test_samples):
    for j in range(canvas_size[1]):
        for c in range(3):
            row_train[i,j,c]=np.sum(x_test[i,j,:,c])


            
col_train=np.zeros((samples-test_samples, canvas_size[0], 3, 1))
for i in range(samples-test_samples):
    for j in range(canvas_size[0]):
        for c in range(3):
            col_train[i,j,c]=np.sum(x_train[i,:,j,c])
            
col_test=np.zeros((test_samples, canvas_size[0], 3, 1))
for i in range(test_samples):
    for j in range(canvas_size[0]):
        for c in range(3):
            col_test[i,j,c]=np.sum(x_test[i,:,j,c])
            
            
print(row_train.shape)

IndexError: too many indices for array: array is 1-dimensional, but 4 were indexed

## Normalization

In [9]:
#x_train = x_train.reshape(x_train.shape[0], canvas_size[0], canvas_size[1],3)
#x_test = x_test.reshape(x_test.shape[0],  canvas_size[0], canvas_size[1],3)

#Normalization
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
y_train = y_train.astype('float32')/max(canvas_size)
y_test = y_test.astype('float32')/max(canvas_size)


#histogram
row_train = row_train.astype('float32')/(canvas_size[0]*255)
row_test = row_test.astype('float32')/(canvas_size[0]*255)
col_train = col_train.astype('float32')/(canvas_size[1]*255)
col_test = col_test.astype('float32')/(canvas_size[1]*255)

NameError: name 'row_test' is not defined

## Define and train model

In [10]:
img_input = keras.Input(shape=(canvas_size[1], canvas_size[0], 3), name="image")
row_input = keras.Input(shape=(canvas_size[1], 3, 1), name="row")
col_input = keras.Input(shape=(canvas_size[0], 3, 1), name="col")

x = layers.MaxPooling2D(pool_size=(2,2))(img_input)
x = layers.Convolution2D(20, (3,3),activation='relu')(x)
x = layers.Convolution2D(15, (3,3),activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Convolution2D(10, (3,3),activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x_flat = layers.Flatten()(x)
r = layers.Flatten()(row_input)
r = layers.Dense(300, activation = 'relu')(r)
r = layers.Dense(50, activation = 'relu')(r)
r_flat = layers.Flatten()(r)
c = layers.Flatten()(col_input)
c = layers.Dense(300, activation = 'relu')(c)
c = layers.Dense(50, activation = 'relu')(c)
c_flat = layers.Flatten()(c)

conc = layers.concatenate([x_flat, r_flat, c_flat])

pos_pred = layers.Dense(4, name="output")(conc)

model = keras.Model(inputs=[img_input, row_input, col_input], outputs=[pos_pred])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.compile(optimizer='adam',loss='mse')


In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 150, 150, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 148, 148, 20) 560         max_pooling2d[0][0]              
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 146, 146, 15) 2715        conv2d[0][0]                     
______________________________________________________________________________________________

In [14]:
history=model.fit(
    {"image": x_train, "row": row_train, "col": col_train},
    {"output": y_train},
    epochs=30, 
    batch_size=20, 
    validation_split=0.1, 
    shuffle=True, 
    verbose=2,
)

NameError: name 'col_train' is not defined